In [1]:
from pyspark.sql import SparkSession

imports the SparkSession class from PySpark, which is used to create a Spark environment for reading, processing, and analyzing data.

In [2]:
spark = SparkSession.builder.appName("HousingRDDExample").getOrCreate()
sc = spark.sparkContext

These statements create a Spark session named `"HousingRDDExample"` to set up the Spark environment, and then access its SparkContext (`sc`) to work with low-level RDD operations.

In [3]:
data = sc.textFile("Housing.csv")

This statement reads the file `"Housing.csv"` as a Resilient Distributed Dataset (RDD) of text lines, allowing low-level Spark operations on each line of the dataset.

In [4]:
header = data.first()
rows = data.filter(lambda line: line != header)

These statements extract the first line of the RDD as the header and then create a new RDD `rows` that excludes the header, leaving only the data rows.

In [5]:
split_rdd = rows.map(lambda line: line.split(","))

splits each line in the rows RDD by commas, creating a new RDD split_rdd where each element is a list of values representing the columns of a row.

In [6]:
housing_rdd = split_rdd.map(lambda x: (int(x[0]), float(x[1]), int(x[2]), int(x[3]), int(x[4]), x[5]))

This statement converts the split_rdd into a structured RDD housing_rdd by mapping each row to a tuple with appropriate data types: id as integer, price as float, area, bedrooms, and bathrooms as integers, and location as string.

In [7]:
print("=== First 10 Rows ===")
for row in housing_rdd.take(10):
    print(row)

=== First 10 Rows ===
(13300000, 7420.0, 4, 2, 3, 'yes')
(12250000, 8960.0, 4, 4, 4, 'yes')
(12250000, 9960.0, 3, 2, 2, 'yes')
(12215000, 7500.0, 4, 2, 2, 'yes')
(11410000, 7420.0, 4, 1, 2, 'yes')
(10850000, 7500.0, 3, 3, 1, 'yes')
(10150000, 8580.0, 4, 3, 4, 'yes')
(10150000, 16200.0, 5, 3, 2, 'yes')
(9870000, 8100.0, 4, 1, 2, 'yes')
(9800000, 5750.0, 3, 2, 4, 'yes')


These statements display a header message and then print the first 10 rows of the structured RDD housing_rdd to show a sample of the dataset.

In [8]:
avg_price_rdd = housing_rdd.map(lambda x: (x[5], (x[1], 1))) \
                           .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
                           .mapValues(lambda v: round(v[0] / v[1], 2))

This statement computes the average house price for each location by summing the prices and counts per location and then dividing the total price by the count, rounding the result to two decimal places.


In [9]:
print("\n=== Average Price ===")
for loc, avg in avg_price_rdd.collect():
    print(loc, avg)


=== Average Price ===
no 3606.44
yes 5404.59


These statements display a header message and then print the average house price for each location by collecting and iterating over the results from avg_price_rdd.

In [10]:
max_area = housing_rdd.map(lambda x: x[1]).max()
print("Maximum area in dataset:", max_area)

Maximum area in dataset: 16200.0


To find the highest house area in the dataset by extracting the area from each row and then print the maximum value.